References:
* https://stackoverflow.com/questions/34523111/the-most-elegant-way-to-get-back-from-pandas-df-dummies
* https://github.com/pandas-dev/pandas/blob/v1.4.0/pandas/core/reshape/reshape.py#L811-L994

In [1]:
import torch
import torch.distributions as tdist
from typing import Any, Callable, Tuple, Union, cast
from torch import Tensor
from torch import nn
from random import random


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
import pickle

In [4]:
german_feature_metadata = pickle.load(open('Data_Sets/German_Credit_Data/german-feature-metadata.p', 'rb'))

In [5]:
from explainers.perturbation_methods import NewDiscrete_NormalPerturbation

## Inspect behavior of perturbation class on an existing dataset with categorical vars

In [6]:
perturbation = NewDiscrete_NormalPerturbation("tabular", mean=0.01,
                                              std_dev=0.01,
                                              flip_percentage=0.5)

In [7]:
# (Train & Test) Loaders
import ML_Models.data_loader as loaders

In [8]:
loader_train, loader_test = loaders.return_loaders(data_name='german', is_tabular=True,
                                                           batch_size=16)
data_iter = iter(loader_train)
inputs, labels = data_iter.next()

In [9]:
x_german = inputs[0]

In [10]:
x_german.shape

torch.Size([60])

In [11]:
labels

tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [12]:
top_k_mask = torch.BoolTensor(np.zeros(x_german.shape).flatten())

In [13]:
x_perturbed = perturbation.get_perturbed_inputs(original_sample=x_german,
                                               feature_mask=top_k_mask,
                                               num_samples=500,
                                               max_distance=0.5,
                                               feature_type=german_feature_metadata['feature_types'],
                                               feature_num_cols=german_feature_metadata['feature_n_cols'])

In [14]:
x_german

tensor([0.1176, 0.0708, 0.3333, 0.3333, 0.1250, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000], dtype=torch.float64)

In [15]:
x_perturbed

tensor([[0.1273, 0.0767, 0.3399,  ..., 0.0000, 0.0000, 1.0000],
        [0.1260, 0.0913, 0.3401,  ..., 0.0000, 0.0000, 1.0000],
        [0.1358, 0.0795, 0.3567,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [0.1325, 0.0748, 0.3434,  ..., 1.0000, 1.0000, 0.0000],
        [0.1236, 0.0894, 0.3434,  ..., 1.0000, 0.0000, 1.0000],
        [0.1220, 0.0799, 0.3448,  ..., 0.0000, 0.0000, 1.0000]],
       dtype=torch.float64)

In [ ]:
import pandas as pd
from tqdm import tqdm

# ANN models
import torch
import ML_Models.ANN.model as model_ann

# (Train & Test) Loaders
import ML_Models.data_loader as loaders

# Explanation Models
from explainers.lime import LIME
from explainers.shap_explainer_captum import SHAPExplainerC
from explainers.grad import Gradient
from explainers.ebp import EBP
from explainers.integrated_gradients import IntegratedGradients
from explainers.guided_backprop import GuidedBackprop
from explainers.smoothgrad import SmoothGrad
from explainers.gradcam import GradCAM
from explainers.guided_gradcam import GuidedGradCAM
from explainers.lrp import LRP
from explainers.input_x_gradient import InputTimesGradient

# Perturbation Methods
import torch.distributions as tdist
# from explainers.perturbation_methods import RandomPerturbation
# from explainers.perturbation_methods import UniformPerturbation
# from explainers.perturbation_methods import BootstrapPerturbation
# from explainers.perturbation_methods import MarginalPerturbation
# from explainers.perturbation_methods import AdversarialPerturbation
# from explainers.perturbation_methods import NormalPerturbation

# Import Evaluation Methods
from evaluator import Evaluator

# lime utils
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json

# torch utils
import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

# experiment parameter defaults
from experiment_config import *

from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

from experiment_utils import *